In [1]:
%%configure -f
{
    "conf":{
        "spark.executor.instances": "4",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "1"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
575,application_1765289937462_0568,pyspark,idle,Link,Link,None,
576,application_1765289937462_0569,pyspark,idle,Link,Link,None,
592,application_1765289937462_0585,pyspark,idle,Link,Link,None,
593,application_1765289937462_0586,pyspark,idle,Link,Link,None,
597,application_1765289937462_0590,pyspark,idle,Link,Link,None,
617,application_1765289937462_0610,pyspark,idle,Link,Link,None,
618,application_1765289937462_0611,pyspark,idle,Link,Link,None,
620,application_1765289937462_0613,pyspark,idle,Link,Link,None,
624,application_1765289937462_0617,pyspark,idle,Link,Link,None,
629,application_1765289937462_0622,pyspark,idle,Link,Link,None,


In [2]:
from pyspark.sql import SparkSession
import csv
from io import StringIO
import time


def parse_csv(line):
    return next(csv.reader([line]))


sc = SparkSession \
    .builder \
    .appName("RDD query 1 execution") \
    .getOrCreate() \
    .sparkContext

rdd1 = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2010_2019.csv")\
            .map(parse_csv)

rdd2 = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2020_2025.csv")\
            .map(parse_csv)

crime_rdd_raw = rdd1.union(rdd2)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
632,application_1765289937462_0625,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
def parse_age(s):
    try:
        age = int(s)
        return age if age > 0 else None
    except:
        return None

def age_bucket(age):
    if age is None:
        return None
    if age < 18:
        return "Children"
    elif age <= 24:
        return "Young Adults"
    elif age <= 64:
        return "Adults"
    else:
        return "Elderly"

start = time.time() 

agg_buckets = (
    crime_rdd_raw
    .filter(lambda r: len(r) > 11 and r[9] and "aggravated assault" in r[9].lower())
    # Convert age from string → int → bucket, then produce (bucket, 1)
    .map(lambda r: (age_bucket(parse_age(r[11])), 1))
    # drop rows with no valid bucket
    .filter(lambda x: x[0] is not None)
    .reduceByKey(lambda a, b: a + b)
    .sortBy(lambda x: x[1], ascending=False)
)



# Collect to the driver to print 
for group, cnt in agg_buckets.collect():
    print(group, cnt)

end = time.time()
print("Execution time:", end - start, "seconds")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Adults 121660
Young Adults 33758
Children 10904
Elderly 6011
Execution time: 17.90143895149231 seconds